# I. Hướng dẫn

## Khởi tạo Spark

In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count,lit

spark = (SparkSession
         .builder
         .appName("Support Vector Machine (SVM)")
         .getOrCreate())

## Đọc và load tập dữ liệu Iris

In [2]:
irisDF = (spark.read
          .option("HEADER", True)
          .option("inferSchema", True)
          .csv("./iris.csv")
         )

irisDF.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

## Chuyển cột `class` (kiểu string) thành `label` (kiểu double)

In [3]:
from pyspark.ml.feature import StringIndexer

class_indexer = StringIndexer(inputCol='class',outputCol='label')

irisDFindexed = class_indexer.fit(irisDF).transform(irisDF)

irisDFindexed.show()

+------------+-----------+------------+-----------+-----------+-----+
|sepal_length|sepal_width|petal_length|petal_width|      class|label|
+------------+-----------+------------+-----------+-----------+-----+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|  0.0|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|  0.0|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|  0.0|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|  0.0|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|  0.0|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|  0.0|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|  0.0|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|  0.0|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|  0.0|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|  0.0|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|  0.0|
|         4.8|      

## Tập dữ liệu Iris

`sepal_length`: chiều dài đài hoa (cm)

`sepal_width`: chiều rộng đài hoa (cm)

`petal_length`: chiều dài cánh hoa (cm)

`petal_width`: chiều rộng cánh hoa (cm)

`class/label`: loại hoa

## Chia dữ liệu thành train/test set

In [4]:
(trainDF, testDF) = irisDFindexed.randomSplit([.8, .2], seed=1)

## Xem các loại biến trong tập dữ liệu

In [5]:
irisDFindexed.dtypes

[('sepal_length', 'double'),
 ('sepal_width', 'double'),
 ('petal_length', 'double'),
 ('petal_width', 'double'),
 ('class', 'string'),
 ('label', 'double')]

## Biến đổi train data và test data theo định dạng của Spark

In [6]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['sepal_length','sepal_width','petal_length','petal_width'],
                            outputCol='features')
assembler_train = assembler.transform(trainDF)

final_train = assembler_train.select('features','label')
final_train.show(3)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[4.3,3.0,1.1,0.1]|  0.0|
|[4.4,2.9,1.4,0.2]|  0.0|
|[4.4,3.0,1.3,0.2]|  0.0|
+-----------------+-----+
only showing top 3 rows



## Sử dụng Linear Support Vector Classifier với One Vs Rest

**1.1** Huấn luyện mô hình trên `final_train` với `LinearSVC` theo cách tiếp cận `OneVsRest` với `labelCol` là `'lable'` và `featuresCol` là `'features'`

In [7]:
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(featuresCol='features', labelCol='label', maxIter=10, regParam=0.1)
ovr = OneVsRest(classifier=lsvc)
ovrModel = ovr.fit(final_train)

**1.2.** Áp dụng mô hình trên test data

Áp dụng biến đổi cho tập test tương tự như trên tập train. In ra vài dòng sau khi biến đổi để xem kết quả.

In [8]:
assembler_test = assembler.transform(testDF)
final_test = assembler_test.select('features','label')
final_test.show(3)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[4.5,2.3,1.3,0.3]|  0.0|
|[4.8,3.1,1.6,0.2]|  0.0|
|[4.8,3.4,1.6,0.2]|  0.0|
+-----------------+-----+
only showing top 3 rows



Dự đoán trên test data

In [9]:
predictions = ovrModel.transform(final_test)
predictions.select("prediction", "label").show()

+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       2.0|  2.0|
|       0.0|  0.0|
|       2.0|  1.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       2.0|  1.0|
|       0.0|  0.0|
|       2.0|  2.0|
|       2.0|  1.0|
|       2.0|  1.0|
|       2.0|  1.0|
|       2.0|  2.0|
|       2.0|  1.0|
|       2.0|  1.0|
+----------+-----+
only showing top 20 rows



**1.3.** Tính giá trị `Accuracy` của mô hình trên tập test

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.730769
Test Error = 0.269231


# II. Áp dụng

## Câu 1 - Áp dụng `LinearSVC` với tập dữ liệu `Auto`

Câu hỏi này sử dụng SVM trên tập dữ liệu `Auto` để dự đoán một xe cho trước có `mpg` là `high` hay `low`.

**1.1.** Tạo một binary variable nhận giá trị 1 (`high`) với các xe có `mpg` lớn hơn median mpg, và nhận giá trị 0 (`low`) cho các xe còn lại.

In [35]:
# Viết code của bạn ở đây
autoDF = (spark.read
          .option("HEADER", True)
          .option("inferSchema", True)
          .csv("./Auto.csv")
         )

autoDF.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- name: string (nullable = true)



In [12]:
from  pyspark.sql import functions 
import numpy as np
from pyspark.sql.types import FloatType

### https://stackoverflow.com/questions/42826502/pyspark-dataframe-group-by-filtering

def median(values):
    try:
        median = np.median(values)
        return round(float(median),2)
    except Exception:
        return None

median = functions.udf(median,FloatType())

In [13]:
autoDF2 = (autoDF.withColumn("temp",functions.lit(1)) 
            .groupBy("temp")
            .agg(functions.collect_list("mpg").alias("mpg"))
          )

autoDF2.show()

+----+--------------------+
|temp|                 mpg|
+----+--------------------+
|   1|[18.0, 15.0, 18.0...|
+----+--------------------+



In [14]:
temp = autoDF2.withColumn("median", median("mpg"))

In [16]:
medianOfMPG = temp.collect()[0]["median"]

# Giá trị median cần tìm
medianOfMPG

23.0

In [37]:
autoDF = autoDF.withColumn("mpgClassifier", functions.when(functions.col("mpg") >= medianOfMPG, 1).otherwise(0))

autoDF.show()

+----+---------+------------+----------+------+------------+----+------+--------------------+-------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|mpgClassifier|
+----+---------+------------+----------+------+------------+----+------+--------------------+-------------+
|18.0|        8|       307.0|       130|  3504|        12.0|  70|     1|chevrolet chevell...|            0|
|15.0|        8|       350.0|       165|  3693|        11.5|  70|     1|   buick skylark 320|            0|
|18.0|        8|       318.0|       150|  3436|        11.0|  70|     1|  plymouth satellite|            0|
|16.0|        8|       304.0|       150|  3433|        12.0|  70|     1|       amc rebel sst|            0|
|17.0|        8|       302.0|       140|  3449|        10.5|  70|     1|         ford torino|            0|
|15.0|        8|       429.0|       198|  4341|        10.0|  70|     1|    ford galaxie 500|            0|
|14.0|        8|       454.0

**1.2.** Áp dụng linear support vector classiﬁer cho tập dữ liệu với các giá trị siêu tham số `regParam` khác nhau để dự đoán `mpg`. Cho biết cross-validation error ứng với các giá trị khác nhau của siêu tham số này. Nhận xét kết quả thu được. Tham khảo document về linear support vector classiﬁer của Spark ở [LinearSVC](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.LinearSVC).

In [39]:
# Viết code của bạn ở đây

#chuyển cột binary value mpgClassifier vừa làm được thành label kiểu double
mpg_indexer = StringIndexer(inputCol='mpgClassifier',outputCol='label')
autoDFindexed = mpg_indexer.fit(autoDF).transform(autoDF)
autoDFindexed.show()

+----+---------+------------+----------+------+------------+----+------+--------------------+-------------+-----+
| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|mpgClassifier|label|
+----+---------+------------+----------+------+------------+----+------+--------------------+-------------+-----+
|18.0|        8|       307.0|       130|  3504|        12.0|  70|     1|chevrolet chevell...|            0|  1.0|
|15.0|        8|       350.0|       165|  3693|        11.5|  70|     1|   buick skylark 320|            0|  1.0|
|18.0|        8|       318.0|       150|  3436|        11.0|  70|     1|  plymouth satellite|            0|  1.0|
|16.0|        8|       304.0|       150|  3433|        12.0|  70|     1|       amc rebel sst|            0|  1.0|
|17.0|        8|       302.0|       140|  3449|        10.5|  70|     1|         ford torino|            0|  1.0|
|15.0|        8|       429.0|       198|  4341|        10.0|  70|     1|    ford galaxie

In [45]:
#Chia dữ liệu thành trainset, 
(trainDF, testDF) = autoDFindexed.randomSplit([.8, .2], seed=1)

print( trainDF.count())
print( testDF.count())

321
76


In [46]:
## Xem các loại biến trong tập dữ liệu
autoDFindexed.dtypes

[('mpg', 'double'),
 ('cylinders', 'int'),
 ('displacement', 'double'),
 ('horsepower', 'string'),
 ('weight', 'int'),
 ('acceleration', 'double'),
 ('year', 'int'),
 ('origin', 'int'),
 ('name', 'string'),
 ('mpgClassifier', 'int'),
 ('label', 'double')]

In [59]:
## Biến đổi train data và test data theo định dạng của Spark
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['cylinders','displacement','weight','acceleration','year','origin'],
                            outputCol='features')
assembler_train = assembler.transform(trainDF)

final_train = assembler_train.select('features','label')
final_train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[8.0,304.0,4732.0...|  1.0|
|[8.0,307.0,4376.0...|  1.0|
|[8.0,360.0,4615.0...|  1.0|
|[8.0,318.0,4382.0...|  1.0|
|[8.0,400.0,4997.0...|  1.0|
|[8.0,429.0,4633.0...|  1.0|
|[8.0,350.0,4456.0...|  1.0|
|[8.0,350.0,4499.0...|  1.0|
|[8.0,383.0,4955.0...|  1.0|
|[8.0,400.0,4906.0...|  1.0|
|[8.0,429.0,4952.0...|  1.0|
|[8.0,455.0,4951.0...|  1.0|
|[8.0,307.0,4098.0...|  1.0|
|[8.0,318.0,3940.0...|  1.0|
|[8.0,350.0,3988.0...|  1.0|
|[8.0,350.0,4055.0...|  1.0|
|[8.0,350.0,4699.0...|  1.0|
|[8.0,350.0,4502.0...|  1.0|
|[8.0,350.0,4274.0...|  1.0|
|[8.0,350.0,4100.0...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [60]:
## Sử dụng Linear Support Vector Classifier với One Vs Rest
### 1.1 Huấn luyện mô hình trên final_train với LinearSVC theo cách tiếp cận OneVsRest với labelCol là 'lable' và featuresCol là 'features'

from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(featuresCol='features', labelCol='label', maxIter=10, regParam=0.1)
ovr = OneVsRest(classifier=lsvc)
ovrModel = ovr.fit(final_train)

In [ ]:
### 1.2. Áp dụng mô hình trên test data

In [61]:
assembler_test = assembler.transform(testDF)
final_test = assembler_test.select('features','label')
final_test.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[8.0,350.0,3664.0...|  1.0|
|[8.0,302.0,3169.0...|  1.0|
|[8.0,302.0,3870.0...|  1.0|
+--------------------+-----+
only showing top 3 rows



In [ ]:
Dự đoán trên test data

In [70]:
predictions = ovrModel.transform(final_test)
predictions.select("prediction", "label").show()

+----------+-----+
|prediction|label|
+----------+-----+
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
|       1.0|  1.0|
+----------+-----+
only showing top 20 rows



In [79]:
predictions.select("prediction", "label").filter(col("prediction") != col("label")).show()

+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       0.0|  1.0|
|       1.0|  0.0|
|       1.0|  0.0|
+----------+-----+



In [ ]:
1.3. Tính giá trị Accuracy của mô hình trên tập test

In [63]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.881579
Test Error = 0.118421


## Câu 2 - So sánh các mô hình phân loại

Thực hiện việc train tất cả các mô hình `LogisticRegression`, `DecisionTreeClassifier` và `RandomForestClassifier`, `GBTClassifier`, `MultilayerPerceptronClassifier`, `LinearSVC`, `NaiveBayes` trên tập một tập dữ liệu cho bài toán phân loại (có từ 10-100 thuộc tính và 100-1000 đối tượng) mà bạn quan tâm ở [UCI Classification Dataset](https://archive.ics.uci.edu/ml/datasets.php?format=&task=cla&att=&area=&numAtt=10to100&numIns=100to1000&type=&sort=nameUp&view=table). Điều chỉnh các siêu tham số của các mô hình để chọn mô hình tốt nhất dùng cross validation. Tham khảo  notebook "10-7 Hyperparameter Tuning" ở sách Learning Spark. So sánh và nhận xét về kết quả của các mô hình. Để tránh lặp lại các bước xử lý giống nhau nhiều lần như ở trên bạn nên tạo pipeline các bước xử lý. Tham khảo cách tạo pipeline cho mô hình ở [đây](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Tham khảo document về các classifier của Spark ở [classification module](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.classification).

In [ ]:
# Viết code của bạn ở đây


## Câu 3 - So sánh các mô hình hồi quy

Thực hiện việc train tất cả các mô hình `LinearRegression`, `DecisionTreeRegression` và `RandomForestRegression`, `GBTRegression` trên tập một tập dữ liệu cho bài toán hồi quy (có từ 10-100 thuộc tính và 100-1000 đối tượng) mà bạn quan tâm ở [UCI Regression Dataset](https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=10to100&numIns=100to1000&type=&sort=nameUp&view=table). Điều chỉnh các siêu tham số của các mô hình để chọn mô hình tốt nhất dùng cross validation. Tham khảo  notebook "10-7 Hyperparameter Tuning" ở sách Learning Spark. So sánh và nhận xét về kết quả của các mô hình. Để tránh lặp lại các bước xử lý giống nhau nhiều lần như ở trên bạn nên tạo pipeline các bước xử lý. Tham khảo cách tạo pipeline cho mô hình ở [đây](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Tham khảo document về các classifier của Spark ở [regression module](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.regression).

In [ ]:
# Viết code của bạn ở đây
